In [1]:
import glob

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

from pynistview_utils import *
from selelems import *

import cv2

plt.style.use('dark_background')

In [2]:
# Specify the number of features - 1 in the image. This is used to segment and create masks.
segments = 5

# Specify gaussian denoising sigma
sigma = 2

# Specify non-local denoising strength h. Larger h -> more denoising.
h = 20

# Specify a scale for drawing the vector arrows. Smaller number -> longer arrow.
arrow_scale = 0.5

# Specify a color for the vector arrows.
#arrow_color = 'black'
arrow_color = 'white'

In [3]:
def process_data(indir, sempa_file_suffix, name, run):
    # Read in image data.

    full_name = name + f'{run:0>3}'

    file = indir + full_name

    files = np.asarray(glob.glob('{}*x*{}'.format(file, sempa_file_suffix)))
    files = np.append(files, glob.glob('{}*y*{}'.format(file, sempa_file_suffix)))
    files = sorted(np.append(files, glob.glob('{}*z*{}'.format(file, sempa_file_suffix))))

    # intensity, _ = image_data(file + im_suffix)
    # m_1, axis_1 = image_data(file + m1_suffix)
    # m_2, axis_2 = image_data(file + m2_suffix)
    intensity, _ = image_data(files[0])
    m_1, axis_1 = image_data(files[2])
    m_2, axis_2 = image_data(files[3])

    # Get file dimensions

    m_1_ydim, m_1_xdim = m_1.shape
    m_2_ydim, m_2_xdim = m_2.shape

    # Extract extrema for later processing

    m_1_min, m_1_max = m_1.min(), m_1.max()
    m_2_min, m_2_max = m_2.min(), m_2.max()

    # Flatten the intensity image

    intensity_blurred = median_filter(intensity, 3);
    intensity_flat = intensity - intensity_blurred;

    scale = get_scale(files[0])

    m_1_delined, m_2_delined = remove_line_errors(m_1, 20), remove_line_errors(m_2, 20)
    m_1_delined, m_2_delined = rescale(m_1_delined, m_1_min, m_1_max), rescale(m_2_delined, m_2_min, m_2_max)

    m_1_shift, m_2_shift = mean_shift_filter(m_1_delined), mean_shift_filter(m_2_delined)

    m_1_thresh, m_1_seg, m_1_masks = segment_image(m_1_shift, segments=segments)
    m_2_thresh, m_2_seg, m_2_masks = segment_image(m_2_shift, segments=segments)

    # m_1 first
    m_1_light_mask, m_1_dark_mask = m_1_masks[0, 0], m_1_masks[0, 1]

    m_1_light_fit_all, m_1_light_fit = fit_image(m_1, m_1_light_mask)
    m_1_dark_fit_all, m_1_dark_fit = fit_image(m_1, m_1_dark_mask)

    m_1_light_fitted, m_1_dark_fitted = m_1 - m_1_light_fit_all, m_1 - m_1_dark_fit_all

    # and m_2
    m_2_light_mask, m_2_dark_mask = m_2_masks[0, 0], m_2_masks[0, 1]

    m_2_light_fit_all, m_2_light_fit = fit_image(m_2, m_2_light_mask)
    m_2_dark_fit_all, m_2_dark_fit = fit_image(m_2, m_2_dark_mask)

    m_2_light_fitted, m_2_dark_fitted = m_2 - m_2_light_fit_all, m_2 - m_2_dark_fit_all


    # Rescale both
    m_1_light_fitted, m_2_light_fitted = rescale_to(m_1_light_fitted, m_1), rescale_to(m_2_light_fitted, m_2)

    m_1_fitted_denoised, m_1_fitted_denoised_blurred = clean_image(m_1_light_fitted, sigma=sigma, h=h)
    m_2_fitted_denoised, m_2_fitted_denoised_blurred = clean_image(m_2_light_fitted, sigma=sigma, h=h)

    # Minimize the min to max spread on the magnitudes of the combines images. Use CoM for a starting point.

    offsets, status, message = find_offsets(m_1_fitted_denoised, m_2_fitted_denoised)
    m_1_offset, m_2_offset = offsets

    # Apply offsets

    m_1_denoised, m_2_denoised = m_1_fitted_denoised - m_1_offset, m_2_fitted_denoised - m_2_offset

    # Extract phase and magnitude information

    phases = get_phases(m_1_denoised, m_2_denoised)
    magnitudes = get_magnitudes(m_1_denoised, m_2_denoised)
    magnitudes /= magnitudes.max()
    print('Phase spectrum for run {}: {:.6f} - {:.6f} ({:.3f}º - {:.3f}º)'.format(
        run, phases.min(), phases.max(), phases.min() * 180 / np.pi, phases.max() * 180 / np.pi))

    # Show a colorized version of the domains

    img = render_phases_and_magnitudes(magnitudes, phases)

    save_file(file, intensity, img, axis_1, axis_2, scale, img_denoised_1=m_1_denoised,
              img_denoised_2=m_2_denoised, arrow_color=arrow_color, arrow_scale=arrow_scale)

In [4]:
# Configure input files

indir = "/Users/rmoraski/Documents/iCloud/School/Electrons/NISTView-ish/"
sempa_file_suffix = 'sempa'

#name = "191001_Fe_whisker/Fe_whisker"
#name = "FeGd_Nov_Dec/171117_FeGd"
#name = "FeGd_Nov_Dec/FeGd100"
#name = "FeGd_Nov_Dec/FeGd200"
name = "20201116_Zeros/SEMPA"

#run = 1

for run in range(1, 6):
    process_data(indir, sempa_file_suffix, name, run)

ValueError: `x0` is infeasible.